# Objetivo del notebook

El notebook actual tiene como objetivo principal, proceder con el desarrollo del modelo final, ya con nuestros datos procesados. El notebook actual comprende unicamente la creacion de un modelo a partir del dataset procesado. No obstante, se plantea tambien el ensamblaje de varios modelos para tratar de garantizar un mejor desempenio del modelo final y, de paso, poner en practica esta caracteristica de construccion de modelos.

# Importar las librerias a utilizar

En las siguientes celdas, se importan todas las librerias externas y metodos especificos que son utilizados a lo largo del notebook. 

In [1]:
# Librerias y metodos para analisis y manipulacion de datos
import numpy as np
import pandas as pd

# Clases y metodos de Sklearn
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report, f1_score, precision_score, roc_curve, RocCurveDisplay
from sklearn.utils import shuffle

# Otras librerias 
import os
import warnings
import random
import joblib
from imblearn.over_sampling import SMOTE

warnings.filterwarnings(action='ignore')

# Carga del dataset procesado

In [2]:
# Defino un objeto DataFrame que cargue en memoria el dataset ya procesado, que generamos en el notebook anterior
dataset__route = "../../data/processed/processed__census_income.csv"
adult_df = pd.read_csv(dataset__route, low_memory=False)

# 5 primeros registros del dataset
adult_df.head()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,workclass_Private,...,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia,class
0,0.027397,0.083004,0.600000,0.0,0.0,0.193878,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1
1,0.328767,0.123678,0.800000,0.0,0.0,0.500000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1
2,0.191781,0.094596,0.533333,0.0,0.0,0.500000,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1
3,0.315068,0.128939,0.733333,0.0,0.0,0.448980,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1
4,0.246575,0.058658,0.266667,0.0,0.0,0.377551,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1


In [3]:
adult_df.shape

(48842, 98)

# Creacion de subconjuntos de entrenamiento, test y validacion

Para la creacion de los subconjuntos de datos, voy a asignar las siguientes medidas:

* Conjunto entrenamiento ==>  80%

* Conjunto testing ==>  20%

***IMPORTANTE***. Es vital que el modelo no sufra desviacion hacia ninguna clase, y sea capaz de clasificar correctamente registros, con buena precision para ambas clases.

Es por esto que, antes de llevar a cabo la creacion de los subconjuntos de datos, me voy a asegurar que, al menos el subconjunto de entrenamiento, tenga clases balanceadas.

In [4]:
X = adult_df.drop(columns = ['class'])
y = adult_df['class']

In [5]:
print('Dimensiones de y ==>', y.shape)

indexes_y = y.value_counts().index
classes_y = y.value_counts().values
print('Ejemplos para cada clase:')
for index, class_ in zip(indexes_y, classes_y):
    print(index, '==>', class_)

Dimensiones de y ==> (48842,)
Ejemplos para cada clase:
1 ==> 37155
0 ==> 11687


In [6]:
# Instancio un objeto de la clase SMOTE para generar nuevos ejemplos de la clase minoritaria
oversample = SMOTE()

X, y = oversample.fit_resample(X, y)

In [7]:
print('Dimensiones de y ==>', y.shape)

indexes_y = y.value_counts().index
classes_y = y.value_counts().values
print('Ejemplos para cada clase:')
for index, class_ in zip(indexes_y, classes_y):
    print(index, '==>', class_)

Dimensiones de y ==> (74310,)
Ejemplos para cada clase:
1 ==> 37155
0 ==> 37155


In [8]:
## Genero los subconjuntos de datos
# Randomizo nuevamente los datos
X, y = shuffle(X, y, random_state = 42)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)
#X_train, X_val, y_train, y_val = train_test_split(X_entrenamiento, y_entrenamiento, test_size=.1)

In [9]:
print("CONJUNTO ENTRENAMIENTO:")
print(X_train.shape, y_train.shape)

CONJUNTO ENTRENAMIENTO:
(59448, 97) (59448,)


In [10]:
#print("CONJUNTO VALIDACION:")
#print(X_val.shape, y_val.shape)

In [11]:
print("CONJUNTO TESTING:")
print(X_test.shape, y_test.shape)

CONJUNTO TESTING:
(14862, 97) (14862,)


## Reviso el balanceo de clases en el conjunto de entrenamiento

In [12]:
indexes_y_train = y_train.value_counts().index
classes_y_train = y_train.value_counts().values
print('Ejemplos para cada clase [ENTRENAMIENTO]:')
for index, class_ in zip(indexes_y_train, classes_y_train):
    print(index, '==>', class_)

Ejemplos para cada clase [ENTRENAMIENTO]:
1 ==> 29729
0 ==> 29719


# Modelo de clasificacion

A continuacion, voy a instanciar un modelo clasificador SVC y optimizar algunos de sus hiperparametros con GridSearchCV.

In [13]:
# Instancio un objeto de la clase SVC
model = SVC()
# Defino un diccionario de parametros a optimizar
grid_params = {
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid']
    }

# Instancio un objeto de la clase GridSearchCV
grid_cv = GridSearchCV(model, grid_params, cv = 6, n_jobs = -1, verbose = 2)

# Entreno el modelo con el conjunto de entrenamiento
grid_cv.fit(X_train, y_train)

Fitting 6 folds for each of 4 candidates, totalling 24 fits


GridSearchCV(cv=6, estimator=SVC(), n_jobs=-1,
             param_grid={'kernel': ['linear', 'poly', 'rbf', 'sigmoid']},
             verbose=2)

In [14]:
# Parametros del mejor modelo
best_params = grid_cv.best_params_

# Mejor modelo entrenado
best_model = grid_cv.best_estimator_

print('Parametros del mejor modelo entrenado ==>', best_params)

Parametros del mejor modelo entrenado ==> {'kernel': 'rbf'}


# Inferencias sobre nuevos ejemplos

Utilizo el mejor modelo para predecir una etiqueta para los ejemplos que el modelo no ha visto durante el entrenamiento, los cuales se encuentran en el conjunto de testing.

In [15]:
## Predicciones del modelo
y_pred = best_model.predict(X_test)

# Primeras 10 etiquetas para el conjunto de testing
print(y_pred[:10])

[1 0 1 1 0 0 0 1 0 0]


# Evaluando el modelo entrenado

In [16]:
print('Accuracy:', best_model.score(X_test, y_test))

Accuracy: 0.831180191091374


In [17]:
# Matriz de confusion y cuadro de metricas
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print(conf_matrix)
print(class_report)

[[6669  767]
 [1742 5684]]
              precision    recall  f1-score   support

           0       0.79      0.90      0.84      7436
           1       0.88      0.77      0.82      7426

    accuracy                           0.83     14862
   macro avg       0.84      0.83      0.83     14862
weighted avg       0.84      0.83      0.83     14862



Si observamos el rendimiento final del modelo mejor entrenado, vemos que el rendimiento que hemos obtenido de extra, en relacion con el propio modelo de base que hicimos, no es relativamente grande. En este aspecto, voy a entrenar diferentes modelos de clasificacion, en diferentes notebooks, y ver si obtenemos un mejor rendimiento con otros de estos modelos.

Tambien podemos ver como el balanceo de clases ha permitido reducir notablemente el subajuste que sufria el modelo al tratar de predecir una instancia perteneciente a la clase minoritaria, que es aquellas mas solvente economicamente.

# Guardo el modelo entrenado

In [18]:
model__route = '../../src/models/'
joblib.dump(best_model, os.path.join(model__route, 'final_model__svc.joblib'))

print('Modelo guardado con exito.')

Modelo guardado con exito.
